In [1]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [4]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver.get(url)
time.sleep(2)

In [5]:
soup=BeautifulSoup(driver.page_source,'html.parser')
trs=soup.select('tr.aos-init')
len(trs)

100

In [7]:
#순위
tr=trs[0]
tr.select_one('.rank').get_text().strip()

'1'

In [8]:
#카테고리
tr.select_one('.category').get_text().strip()

'[음악/댄스/가수]'

In [10]:
#채널
tr.select_one('.subject a').get_text().strip()

'BLACKPINK'

In [11]:
#구독자,조회,비디오 개수
subscriber=tr.select_one('.subscriber_cnt').get_text().strip()
view=tr.select_one('.view_cnt').get_text().strip()
video=tr.select_one('.video_cnt').get_text().strip()

subscriber,view,video

('7390만', '235억0126만', '394개')

In [12]:
lines = []
for tr in trs:
  rank=tr.select_one('.rank').get_text().strip()
  category=tr.select_one('.category').get_text().strip()
  subject=tr.select_one('.subject a').get_text().strip()
  subscriber=tr.select_one('.subscriber_cnt').get_text().strip()
  view=tr.select_one('.view_cnt').get_text().strip()
  video=tr.select_one('.video_cnt').get_text().strip()
  lines.append({'순위':rank,'카테고리':category,'채널':subject,'구독자':subscriber,'조회수':view,'비디오 개수':video})

In [14]:
df=pd.DataFrame(lines)
df

,순위,카테고리,채널,구독자,조회수,비디오 개수
0,1,[음악/댄스/가수],BLACKPINK,7390만,235억0126만,394개
1,2,[음악/댄스/가수],BANGTANTV,6590만,159억7409만,"1,689개"
2,3,[음악/댄스/가수],HYBE LABELS,6520만,223억8194만,853개
3,4,[음악/댄스/가수],SMTOWN,3030만,244억0845만,"3,901개"
4,5,[키즈/어린이],Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개
...,...,...,...,...,...,...
95,96,[미분류],MayTree,421만,7억8409만,174개
96,97,[키즈/어린이],로미유 스토리[Romiyu Story],420만,11억1500만,434개
97,98,[음식/요리/레시피],[Dorothy]도로시,417만,10억1293만,916개
98,99,[취미/라이프],JaeYeol ASMR 재열,412만,11억7858만,"1,092개"


In [20]:
def convert_unit(s):
  s=s.replace('억','').replace('개','').replace(',','').replace('만','0000')
  return f'{int(s):,d}'  # d 숫자 3개마다 , 찍어줌

In [16]:
convert_unit('11억7858만개')

'1,178,580,000'

In [21]:
for column in df.columns[-3:]:
  df[column]=df[column].apply(convert_unit)
df.tail(2)

,순위,카테고리,채널,구독자,조회수,비디오 개수
98,99,[취미/라이프],JaeYeol ASMR 재열,"4,120,000","1,178,580,000","1,092"
99,100,[TV/방송],채널 십오야,"4,070,000","893,020,000",556


In [22]:
from tqdm.notebook import tqdm

In [23]:
lines=[]
for page in tqdm(range(1,11)):
  url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
  driver.get(url)
  time.sleep(2)
  soup=BeautifulSoup(driver.page_source,'html.parser')
  trs=soup.select('tr.aos-init')
  for tr in trs:
    rank=tr.select_one('.rank').get_text().strip()
    category=tr.select_one('.category').get_text().strip()
    subject=tr.select_one('.subject a').get_text().strip()
    subscriber=tr.select_one('.subscriber_cnt').get_text().strip()
    view=tr.select_one('.view_cnt').get_text().strip()
    video=tr.select_one('.video_cnt').get_text().strip()
    lines.append({'순위':rank,'카테고리':category,'채널':subject,'구독자':subscriber,'조회수':view,'비디오 개수':video})

  0%|          | 0/10 [00:00<?, ?it/s]

In [26]:
df=pd.DataFrame(lines)
for column in df.columns[-3:]:
  df[column]=df[column].apply(convert_unit)
df.tail(2)

,순위,카테고리,채널,구독자,조회수,비디오 개수
998,999,[음악/댄스/가수],온스테이지ONSTAGE,"590,000","260,900,000","2,447"
999,998,[TV/방송],레이디액션,"590,000","365,130,000",428


In [27]:
df.to_csv('youtube랭킹.csv',index=False)

In [ ]:
driver.close()